In [ ]:
!pip install pandas transformers guidance numpy evaluate rouge-score nltk py-rouge accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.4/234.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 k

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import json
import nltk
import numpy as np
from guidance import gen
from evaluate import load
import csv
import nltk.translate.bleu_score as bleu
from rouge import Rouge
from transformers import BertTokenizer, BertModel
import torch
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import warnings
from typing import Any, Dict, Tuple
import re

warnings.filterwarnings("ignore", category=UserWarning, message="A new version of the following files was downloaded from")

In [1]:
from google.colab import drive
drive.mount('/content/drive')


MessageError: Error: credential propagation was unsuccessful

# Load Dataset

- id: unique identifier of the example
- source: original model input for paraphrase generation
- hyp1: first alternative paraphrase of the source
- hyp2: second alternative paraphrase of the source
- label: hyp1 or hyp2, based on which of those has been annotated as hallucination
- type: hallucination category assigned.
  - Possible values: addition, named-entity, number, conversion, date, tense, negation, gender, pronoun, antonym, natural

## Trial dataset

This is a small list of examples, provided to help the participants get familiar with the task. Each example contains the following fields:
- id
- source
- hyp1
- hyp2
- label
- type

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [ ]:
from datasets import load_dataset
#load the test data for the cross-model evaluation in both English and Swedish
data = load_dataset("Eloquent/HalluciGen-PG", "cross_model_evaluation")


print(data.keys())

data_df_en = data['cross_model_evaluation_english'].to_pandas()
data_df_se= data['cross_model_evaluation_swedish'].to_pandas()


dict_keys(['cross_model_evaluation_swedish', 'cross_model_evaluation_english'])


In [ ]:
data_df_en

,id,source,hyp1,hyp2
0,0,I would also like to draw attention to Amendme...,the speaker wants to highlight amendments nos...,the speaker mentioned some random amendments ...
1,1,"It is we politicians who are most concerned, b...",Politicians are the most concerned because the...,Politicians are not really worried about the r...
2,2,An important factor here is that the pay and o...,It is crucial that the salary and benefits of ...,One key aspect to consider is that the compens...
3,3,There are four points in relation to which the...,The Committee on Agriculture and Rural Develop...,The Committee on Agriculture and Rural Develop...
4,4,People would think we were mad if we said tomo...,If we were to announce tomorrow that drivers c...,It would be considered normal if we suddenly d...
...,...,...,...,...
590,590,"The Member States are, in other words, being a...",The Member States are being asked to harmonize...,The Member States are being asked to harmonise...
591,591,We struggle with water on a daily basis in the...,"In the Netherlands, water is a daily struggle,...","Water is readily available in the Netherlands,..."
592,592,"On the contrary, things came to a head in the ...","In contrast, the issue escalated during the cr...","Conversely, the situation reached a critical p..."
593,593,It is extremely important that the forthcoming...,It is imperative that the upcoming European Co...,It is crucial that the upcoming European Counc...


In [ ]:
data_df_se

,id,source,hyp1,hyp2
0,0,Romney tillkännager Paul Ryan som vicepresiden...,romney utser paul ryan som sin running mate i...,romney offentliggör att paul ryan blir hans m...
1,1,Syriens militärpolischef hoppade av till oppos...,Syriens militärpolischef hoppade av till oppos...,Syriens militärpolischef hoppade av till oppos...
2,2,"Yahoo går med på att köpa Tumblr för 1,1 milja...",Yahoo har för avsikt att sälja Tumblr till en ...,Yahoo har avtalat att förvärva Tumblr för en s...
3,3,"Önationen rapporterade 65 nya fall den 22 maj,...",Taiwans epidemi har inte sett någon ökning av ...,Taiwans epidemi har sett en ökning av 65 nya f...
4,4,Nordkorea säger att en amerikansk medborgare s...,Nordkoreas ledare Kim Jong-un säger att en ame...,Nordkoreas ledare Kim Jong-un säger att en ame...
...,...,...,...,...
375,375,Påven uppmanar till åtgärder mot klimatförändr...,Påven uppmanar till åtgärder mot klimatförändr...,Påven uppmanar till åtgärder mot klimatförändr...
376,376,"Andy Murray förtjänaren adelstitel, säger Davi...","Andy Murray förtjänaren adelstitel, säger Davi...","Andy Murray förtjänaren adelstitel, säger Davi..."
377,377,Beväpnad man dödar 6 i skottlossning mot Wisco...,En ensam gärningsman har skjutit sex personer ...,En beväpnad individ har dödat sex personer i e...
378,378,Ska jag skriva på det här papperet? Ska jag fö...,Om man skriver på det här papperet och förneka...,Det är inte alltid rätt att säga att någon som...


In [ ]:
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/
  Using cached bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl (119.8 MB)


In [ ]:
!pip install -U "transformers==4.38.1" --upgrade
!pip install accelerate

# Load the model

In [ ]:
#load model Gemma

from transformers import AutoTokenizer, pipeline
import torch

model = "google/gemma-7b-it"
model_id = "google/gemma-7b-it"
dtype = torch.bfloat16

# use quantized model
pipeline = pipeline(
    "text-generation",
    model=model,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True}
    },
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

# Prompt Version 1

In [ ]:
def detect_hallu(source, hyp1, hyp2):
   # Generate the prompt

   answer_format = {"label": ""}

   user_prompt = f'''
   <start_of_turn>user
   Given a "src" and two hypotheses "hyp1" and "hyp2" your task is to detect which of the two hypotheses ("label") is not supported by the source.
   Provide the result in the following format: {answer_format}.

   Src: {source}
   hyp1 : {hyp1}
   hyp2 : {hyp2}

   <end_of_turn>

   Result:
   <start_of_turn>model
   '''

   messages = [
      #{"role": "user", "content": "Imagine you are a classifier"},
      {"role": "user", "content": user_prompt} ]

   prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

   terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

   outputs = pipeline(
        prompt,
        max_new_tokens=150,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
   print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
   hallu_label = outputs[0]["generated_text"][len(prompt):]


   return hallu_label

In [ ]:
def detect_hallu_se(source, hyp1, hyp2):
   # Generate the prompt

   answer_format = {"label": ""}

   user_prompt = f'''
   <start_of_turn>user
   Givet en ”src” och två hypoteser ”hyp1” och ”hyp2” är din uppgift att upptäcka vilken av de två hypoteserna (”label”) som inte stöds av källan.
   Ge resultatet i följande format: {answer_format}.

   Src: {source}
   hyp1 : {hyp1}
   hyp2 : {hyp2}

   <end_of_turn>

   Resultat:
   <start_of_turn>model
   '''

   messages = [
      #{"role": "user", "content": "Imagine you are a classifier"},
      {"role": "user", "content": user_prompt} ]

   prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

   terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

   outputs = pipeline(
        prompt,
        max_new_tokens=150,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
   print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
   hallu_label = outputs[0]["generated_text"][len(prompt):]


   return hallu_label

In [ ]:
import json
import re

def extract_json_data(result_text):

    print(result_text)
    # Find the start and end indices of the JSON data
    start_index = result_text.find('{')
    end_index = result_text.find('}') + 1  # Add 1 to include the closing brace

    # Extract the JSON data from the string
    result_text = result_text[start_index:end_index]
    #print("result text: ", result_text)

    # Replace single quotes used as delimiters with double quotes
    result_text = json.loads(result_text.replace("'", "\""))
    print("result_text:",result_text)
    return result_text

## Generate dataset

In [ ]:
import os
import numpy as np
import pandas as pd

#save results
def save_results(detect_hallu, df, csv_filename):
    skipped_rows = []
    # Split DataFrame into chunks of 10 rows
    chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)

    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']
            hyp1 = row["hyp1"]
            hyp2 = row["hyp2"]

            try:
                # Generate translation
                result = detect_hallu(source, hyp1, hyp2)

                print(result)
                extracted_hallu = extract_json_data(result)

                hallu_label = extracted_hallu["label"]


                # Update DataFrame with new values
                chunk.at[index, "prediction"] = hallu_label
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                skipped_rows.append(index)
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)

        # Load CSV file
        df = pd.read_csv(csv_filename)
        #display(df)
    return skipped_rows

# Generate predictions on the test data

## English

In [ ]:
display(data_df_en)

,id,source,hyp1,hyp2
0,0,I would also like to draw attention to Amendme...,the speaker wants to highlight amendments nos...,the speaker mentioned some random amendments ...
1,1,"It is we politicians who are most concerned, b...",Politicians are the most concerned because the...,Politicians are not really worried about the r...
2,2,An important factor here is that the pay and o...,It is crucial that the salary and benefits of ...,One key aspect to consider is that the compens...
3,3,There are four points in relation to which the...,The Committee on Agriculture and Rural Develop...,The Committee on Agriculture and Rural Develop...
4,4,People would think we were mad if we said tomo...,If we were to announce tomorrow that drivers c...,It would be considered normal if we suddenly d...
...,...,...,...,...
590,590,"The Member States are, in other words, being a...",The Member States are being asked to harmonize...,The Member States are being asked to harmonise...
591,591,We struggle with water on a daily basis in the...,"In the Netherlands, water is a daily struggle,...","Water is readily available in the Netherlands,..."
592,592,"On the contrary, things came to a head in the ...","In contrast, the issue escalated during the cr...","Conversely, the situation reached a critical p..."
593,593,It is extremely important that the forthcoming...,It is imperative that the upcoming European Co...,It is crucial that the upcoming European Counc...


In [ ]:
source = "In May 1998, the Single Market Council adopted provisions regarding the development of a single market in medicinal products"
hyp1= "The Single Market Council adopted provisions regarding the development of a single market in medicinal products in May 1998"
hyp2="The Single Market Council adopted provisions regarding the development of a single market in medicinal products in January 1998"
result = detect_hallu(source,hyp1,hyp2)
print()
print(result)

result = extract_json_data(result)
print(result)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Gen text:  ```python
result = {'label': 'hyp2'}

src = "In May 1998, the Single Market Council adopted provisions regarding the development of a single market in medicinal products"

hyp1 = "The Single Market Council adopted provisions regarding the development of a single market in medicinal products in May 1998"

hyp2 = "The Single Market Council adopted provisions regarding the development of a single market in medicinal products in January 1998"

if hyp2 not in src:
    result['label'] = 'hyp2'

print(result)
```

**Output:**

```
{'label': 'hyp2'}
```

**Explanation:**

* The source text

```python
result = {'label': 'hyp2'}

src = "In May 1998, the Single Market Council adopted provisions regarding the development of a single market in medicinal products"

hyp1 = "The Single Market Council adopted provisions regarding the development of a single market in medicinal products in May 1998"

hyp2 = "The Single Market Council adopted provisions regarding the development of a single ma

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/cross_model_detection_task/cross_model_detection_task_gemma/results_gemma_cross_model_en_v1.csv"

save_results(detect_hallu, data_df_en, csv_filename)

NameError: name 'save_results' is not defined

In [ ]:
# save a csv file in the task format

import pandas as pd

# Load the CSV file
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/cross_model_detection_task/cross_model_detection_task_gemma/results_gemma_cross_model_en_v1.csv")

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'prediction']]
df = df.rename(columns={'prediction': 'label'})

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results//cross_model_detection_task/final_gemma_en_v1_cross_model.csv', index=False)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/cross_model_detection_task/cross_model_detection_task_gemma/results_gemma_cross_model_se_v1.csv"

save_results(detect_hallu_se, data_df_se, csv_filename)

Gen text:  {'label': 'hyp2'}

Källan säger att Romney tillkännager Paul Ryan som vicepresidentkandidat. Det säger inte något om Romney's medkandidat i presidentvalet, så hyp2 är inte stödd av källan.
{'label': 'hyp2'}

Källan säger att Romney tillkännager Paul Ryan som vicepresidentkandidat. Det säger inte något om Romney's medkandidat i presidentvalet, så hyp2 är inte stödd av källan.
{'label': 'hyp2'}

Källan säger att Romney tillkännager Paul Ryan som vicepresidentkandidat. Det säger inte något om Romney's medkandidat i presidentvalet, så hyp2 är inte stödd av källan.
result_text: {'label': 'hyp2'}
Gen text:  ```python
{'label': 'hyp2'}

Src: Syriens militärpolischef hoppade av till opposition
hyp1 : Syriens militärpolischef hoppade av till opposition, vilket minskade intäkterna från mjukvarulicenser med 21 procent till 107,6 miljoner dollar.
hyp2 : Syriens militärpolischef hoppade av till opposition, vilket ökade intäkterna från mjukvarulicenser med 21 procent till 107,6 miljoner d

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  ```python
{'label': 'hyp2'}

Src: En flicka som leker i en hög med färgglada bollar.

hyp1 : Leksaken blir mer upphetsad när bollarna är mindre.
hyp2 : Leksaken leker på bollarna och blir mer upphetsad när bollarna är färgglada.

Resultat:
{'label': 'hyp2'}
```

Hyp2 är inte stödd av källan. Källan visar en flicka som leker på bollarna och blir mer upphetsad när bollarna är färgglada. Men det inte en hypotes om leksaken blir mer upphetsad när bollarna
```python
{'label': 'hyp2'}

Src: En flicka som leker i en hög med färgglada bollar.

hyp1 : Leksaken blir mer upphetsad när bollarna är mindre.
hyp2 : Leksaken leker på bollarna och blir mer upphetsad när bollarna är färgglada.

Resultat:
{'label': 'hyp2'}
```

Hyp2 är inte stödd av källan. Källan visar en flicka som leker på bollarna och blir mer upphetsad när bollarna är färgglada. Men det inte en hypotes om leksaken blir mer upphetsad när bollarna
```python
{'label': 'hyp2'}

Src: En flicka som leker i en hög med färgglada 

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  ```python
{'label': 'hyp2'}

Src: Påven uppmanar till åtgärder mot klimatförändringar i utkastet till encyklika.

hyp1 :  the pope urges action on climate change in the encyclical draft. (the phrasing is close to the original and retains the key message.)
hyp2 :  in the encyclical draft, his holiness the pope advocates for taking steps to address environmental degradation and global warming. (this phrasing significantly rewords the original while conveying the same core idea.)

Resultat:
{'label': 'hyp2'}
```

**Resultat:**

Hypotesen som inte stöds av källan är hyp2. Källan säger att
```python
{'label': 'hyp2'}

Src: Påven uppmanar till åtgärder mot klimatförändringar i utkastet till encyklika.

hyp1 :  the pope urges action on climate change in the encyclical draft. (the phrasing is close to the original and retains the key message.)
hyp2 :  in the encyclical draft, his holiness the pope advocates for taking steps to address environmental degradation and global warming. (t

...

Gen text:  ```
{'label': 'hyp2'}

Src: "Och nu är det vad han vill säga", sa Alesha Badgley, socialchef för Stone County Nursing and Rehabilitation Center, denna vecka.
hyp1 : Denna vecka uttryckte Alesha Badgley, socialchef för Stone County Nursing and Rehabilitation Center, vad han vill säga.
hyp2 : Alesha Badgley, socialchef för Stone County Nursing and Rehabilitation Center, sa något helt annat denna vecka.

Resultat:
{'label': 'hyp2'}
```

**Resultat:**

Hypotesen "hyp2" är inte stödd av källan. I källan sa Alesha Badgley vad han vill säga, men det är inte något
```
{'label': 'hyp2'}

Src: "Och nu är det vad han vill säga", sa Alesha Badgley, socialchef för Stone County Nursing and Rehabilitation Center, denna vecka.
hyp1 : Denna vecka uttryckte Alesha Badgley, socialchef för Stone County Nursing and Rehabilitation Center, vad han vill säga.
hyp2 : Alesha Badgley, socialchef för Stone County Nursing and Rehabilitation Center, sa något helt annat denna vecka.

Resultat:
{'label': '

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'label': 'hyp2'}

Källan stöds inte hyp2, utan stöds hyp1.
{'label': 'hyp2'}

Källan stöds inte hyp2, utan stöds hyp1.
{'label': 'hyp2'}

Källan stöds inte hyp2, utan stöds hyp1.
result_text: {'label': 'hyp2'}


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  ```python
{'label': 'hyp2'}

Src: Uppdaterad - Två explosioner nära mållinjen i Boston Marathon.
hyp1 : Uppdaterad - Två explosioner nära mållinjen i Boston Marathon. Intäkterna från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 21 procent till ett belopp av 107,6 miljoner dollar.
hyp2 : Uppdaterad - Två explosioner nära mållinjen i Boston Marathon. Intäkterna från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 42 procent till
```python
{'label': 'hyp2'}

Src: Uppdaterad - Två explosioner nära mållinjen i Boston Marathon.
hyp1 : Uppdaterad - Två explosioner nära mållinjen i Boston Marathon. Intäkterna från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 21 procent till ett belopp av 107,6 miljoner dollar.
hyp2 : Uppdaterad - Två explosioner nära mållinjen i Boston Marathon. Intäkterna från programvarulicenser, en metrik som noggrant öve

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/cross_model_detection_task/cross_model_detection_task_gemma/results_gemma_cross_model_se_v1.csv")

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'prediction']]
df = df.rename(columns={'prediction': 'label'})

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results//cross_model_detection_task/final_gemma_se_v1_cross_model.csv', index=False)